In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "F:/Data/Full Stack Data Scientist Bootcamp/NLP/fra-eng/fra.txt"


## Prepare the data

In [3]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
    
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)


In [4]:
input_texts ,target_texts ,input_characters ,target_characters 

(['Go.',
  'Go.',
  'Go.',
  'Go.',
  'Hi.',
  'Hi.',
  'Run!',
  'Run!',
  'Run!',
  'Run!',
  'Run!',
  'Run!',
  'Run!',
  'Run!',
  'Run.',
  'Run.',
  'Run.',
  'Run.',
  'Run.',
  'Run.',
  'Run.',
  'Run.',
  'Who?',
  'Wow!',
  'Wow!',
  'Wow!',
  'Duck!',
  'Duck!',
  'Duck!',
  'Fire!',
  'Help!',
  'Hide.',
  'Hide.',
  'Jump!',
  'Jump.',
  'Stop!',
  'Stop!',
  'Stop!',
  'Wait!',
  'Wait!',
  'Wait!',
  'Wait.',
  'Wait.',
  'Wait.',
  'Wait.',
  'Begin.',
  'Begin.',
  'Go on.',
  'Go on.',
  'Go on.',
  'Hello!',
  'Hello!',
  'Hello.',
  'Hello.',
  'Hello.',
  'Hello.',
  'I see.',
  'I see.',
  'I try.',
  'I won!',
  'I won!',
  'I won.',
  'Oh no!',
  'Relax.',
  'Relax.',
  'Relax.',
  'Relax.',
  'Relax.',
  'Relax.',
  'Relax.',
  'Relax.',
  'Relax.',
  'Relax.',
  'Relax.',
  'Relax.',
  'Smile.',
  'Smile.',
  'Smile.',
  'Sorry?',
  'Attack!',
  'Attack!',
  'Attack!',
  'Attack!',
  'Buy it.',
  'Buy it.',
  'Buy it.',
  'Buy it.',
  'Cheers!',
  'Cheers!',

In [5]:
len(input_texts) ,len(target_texts) ,len(input_characters) ,len(target_characters) 

(10000, 10000, 70, 93)

In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

# max_encoder_seq_length = max([len(txt) for txt in input_texts])
# max_decoder_seq_length = max([len(txt) for txt in target_texts])
max_encoder_seq_length = 0
for txt in input_texts:
    print(txt)
    length = len(txt)
    print(length)
    if length > max_encoder_seq_length:
        max_encoder_seq_length = length

print("\n++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")        
max_decoder_seq_length = 0
for txt in target_texts:
    print(txt)
    length = len(txt)
    print(length)
    if length > max_decoder_seq_length:
        max_decoder_seq_length = length


Go.
3
Go.
3
Go.
3
Go.
3
Hi.
3
Hi.
3
Run!
4
Run!
4
Run!
4
Run!
4
Run!
4
Run!
4
Run!
4
Run!
4
Run.
4
Run.
4
Run.
4
Run.
4
Run.
4
Run.
4
Run.
4
Run.
4
Who?
4
Wow!
4
Wow!
4
Wow!
4
Duck!
5
Duck!
5
Duck!
5
Fire!
5
Help!
5
Hide.
5
Hide.
5
Jump!
5
Jump.
5
Stop!
5
Stop!
5
Stop!
5
Wait!
5
Wait!
5
Wait!
5
Wait.
5
Wait.
5
Wait.
5
Wait.
5
Begin.
6
Begin.
6
Go on.
6
Go on.
6
Go on.
6
Hello!
6
Hello!
6
Hello.
6
Hello.
6
Hello.
6
Hello.
6
I see.
6
I see.
6
I try.
6
I won!
6
I won!
6
I won.
6
Oh no!
6
Relax.
6
Relax.
6
Relax.
6
Relax.
6
Relax.
6
Relax.
6
Relax.
6
Relax.
6
Relax.
6
Relax.
6
Relax.
6
Relax.
6
Smile.
6
Smile.
6
Smile.
6
Sorry?
6
Attack!
7
Attack!
7
Attack!
7
Attack!
7
Buy it.
7
Buy it.
7
Buy it.
7
Buy it.
7
Cheers!
7
Cheers!
7
Cheers!
7
Cheers!
7
Eat it.
7
Eat it.
7
Get up.
7
Get up.
7
Get up.
7
Go now.
7
Go now.
7
Go now.
7
Got it!
7
Got it!
7
Got it!
7
Got it?
7
Got it?
7
Got it?
7
Hop in.
7
Hop in.
7
Hug me.
7
Hug me.
7
I fell.
7
I fell.
7
I fled.
7
I knit.
7
I know.
7
I left.
7
I left

17
	Je mange des haricots.

24
	Je mange du pain.

19
	Je mange des fruits.

22
	Je me suis entraîné.

22
	Je me suis entraînée.

23
	Je me sens déprimé.

21
	Je me sens déprimée.

22
	J'ai froid.

13
	Je me sens bien.

18
	Je me sens bien.

18
	Je me sens perdu.

19
	Je me sens perdue.

20
	Je me sens en sécurité.

25
	Je ne me sens pas bien.

25
	Je me sens malade.

20
	Je me sens mal.

17
	Je me sens faible.

20
	Je me sens bien.

18
	Je suis tombée.

17
	Je suis tombé.

16
	Je suis tombé à terre.

24
	Je suis tombée à terre.

25
	Je me suis senti idiot.

25
	Je me suis sentie idiote.

27
	J'ai ressenti de la peur.

27
	Je me sentais bien.

21
	Je me suis senti en sécurité.

31
	Je me suis sentie en sécurité.

32
	J'ai viré Tom.

16
	Je t'ai trouvé.

17
	Je vous ai trouvés.

21
	Je vous ai trouvées.

22
	J'ai commencé à m'ennuyer.

28
	J'ai été pris de vertiges.

28
	J'ai été prise de vertiges.

29
	On m'a mis une amende.

24
	On m'a collé une amende.

26
	J'ai pris une prune.

22
	

In [7]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 14
Max sequence length for outputs: 59


In [8]:
num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length

(70, 93, 14, 59)

## Input Tokens

In [9]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [10]:
# input_token_index1 = {}
# index = 0
# for char in sorted(set(input_characters)):
#     if char not in input_token_index1:
#         input_token_index1[char] = index
#         index += 1

# target_token_index1 = {}
# index = 0
# for char in sorted(set(target_characters)):
#     if char not in target_token_index1:
#         target_token_index1[char] = index
#         index += 1


In [11]:
# input_token_index,input_token_index1

In [12]:
# target_token_index,target_token_index1

In [13]:
input_token_index,target_token_index

({' ': 0,
  '!': 1,
  '"': 2,
  '$': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  ',': 7,
  '-': 8,
  '.': 9,
  '0': 10,
  '1': 11,
  '2': 12,
  '3': 13,
  '5': 14,
  '7': 15,
  '8': 16,
  '9': 17,
  ':': 18,
  '?': 19,
  'A': 20,
  'B': 21,
  'C': 22,
  'D': 23,
  'E': 24,
  'F': 25,
  'G': 26,
  'H': 27,
  'I': 28,
  'J': 29,
  'K': 30,
  'L': 31,
  'M': 32,
  'N': 33,
  'O': 34,
  'P': 35,
  'Q': 36,
  'R': 37,
  'S': 38,
  'T': 39,
  'U': 40,
  'V': 41,
  'W': 42,
  'Y': 43,
  'a': 44,
  'b': 45,
  'c': 46,
  'd': 47,
  'e': 48,
  'f': 49,
  'g': 50,
  'h': 51,
  'i': 52,
  'j': 53,
  'k': 54,
  'l': 55,
  'm': 56,
  'n': 57,
  'o': 58,
  'p': 59,
  'q': 60,
  'r': 61,
  's': 62,
  't': 63,
  'u': 64,
  'v': 65,
  'w': 66,
  'x': 67,
  'y': 68,
  'z': 69},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  '(': 7,
  ')': 8,
  ',': 9,
  '-': 10,
  '.': 11,
  '0': 12,
  '1': 13,
  '2': 14,
  '3': 15,
  '5': 16,
  '8': 17,
  '9': 18,
  ':': 19,
  '?': 20,
  'A': 21,
  'B'

## Encoder and Decoder

In [14]:
# refer to blog for more on the parameters: https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

In [15]:
encoder_input_data,decoder_input_data,decoder_target_data

(array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],


In [16]:
encoder_input_data.shape, decoder_input_data.shape, decoder_target_data.shape

((10000, 14, 70), (10000, 59, 93), (10000, 59, 93))

In [17]:
encoder_input_data[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0

## One-Hot Representation

In [18]:
#You can simply use the keras One-hot encoder to do same thing: https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

## Build the model

In [19]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens)) #encoder input shape
encoder = keras.layers.LSTM(latent_dim, return_state=True) #specify the latent_dim(your timestamp)
encoder_outputs, state_h, state_c = encoder(encoder_inputs) #encoder returns encoder output, hidden state and cell state

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c] #only keep the hidden state and cell state


# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax") #adding dense layer
decoder_outputs = decoder_dense(decoder_outputs) #getting all the output

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


## Train the model

In [20]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=20,
    validation_split=0.2,
)
# Save model
model.save("seq2seq_model")


Epoch 1/20
125/125 [==============================] - 76s 576ms/step - loss: 1.2108 - accuracy: 0.7332 - val_loss: 1.0815 - val_accuracy: 0.7017
Epoch 2/20
125/125 [==============================] - 67s 540ms/step - loss: 0.9351 - accuracy: 0.7479 - val_loss: 0.9581 - val_accuracy: 0.7254
Epoch 3/20
125/125 [==============================] - 70s 565ms/step - loss: 0.8460 - accuracy: 0.7667 - val_loss: 0.8694 - val_accuracy: 0.7488
Epoch 4/20
125/125 [==============================] - 69s 551ms/step - loss: 0.7469 - accuracy: 0.7909 - val_loss: 0.7728 - val_accuracy: 0.7765
Epoch 5/20
125/125 [==============================] - 77s 619ms/step - loss: 0.6736 - accuracy: 0.8065 - val_loss: 0.7219 - val_accuracy: 0.7930
Epoch 6/20
125/125 [==============================] - 73s 582ms/step - loss: 0.6322 - accuracy: 0.8162 - val_loss: 0.6871 - val_accuracy: 0.8004
Epoch 7/20
125/125 [==============================] - 80s 639ms/step - loss: 0.6038 - accuracy: 0.8228 - val_loss: 0.6622 - val_ac

INFO:tensorflow:Assets written to: seq2seq_model\assets


In [23]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("seq2seq_model") #reading saved model

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence



In [24]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


1/1 [==============================] - 0s 31ms/step
-
Input sentence: Go.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 25ms/step
-
Input sentence: Go.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 33ms/step
-
Input sentence: Go.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Go.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 19ms/step
-
Input sentence: Hi.
Decoded sentence: Tom a terait.

1/1 [==============================] - 0s 32ms/step
-
Input sentence: Hi.
Decoded sentence: Tom a terait.

1/1 [==============================] - 0s 31ms/step
-
Input sentence: Run!
Decoded sentence: Attendez !

1/1 [==============================] - 0s 31ms/step
-
Input sentence: Run!
Decoded sentence: Attendez !

1/1 [==============================] - 0s 45ms/step
-
Input sentence: Run!
Decoded sentence: Attendez !

1/1 [==============================] - 0s 31ms/step
-
I

1/1 [==============================] - 0s 31ms/step
-
Input sentence: Run!
Decoded sentence: Attendez !

1/1 [==============================] - 0s 36ms/step
-
Input sentence: Run!
Decoded sentence: Attendez !

1/1 [==============================] - 0s 26ms/step
-
Input sentence: Run!
Decoded sentence: Attendez !

1/1 [==============================] - 0s 31ms/step
-
Input sentence: Run.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 31ms/step
-
Input sentence: Run.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Run.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 22ms/step
-
Input sentence: Run.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 60ms/step
-
Input sentence: Run.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 42ms/step
-
Input sentence: Run.
Decoded sentence: Attendez !

